In [1]:
import intake
import xarray as xr 
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from calendar import monthrange
import numpy as np
import cartopy
from tqdm import tqdm
from scipy import interpolate
import pandas as pd
import seaborn as sns
import os
from scipy import stats

#%config InlineBackend.figure_format='retina'

from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [2]:
parent_dir = '/home/jovyan/PlanetaryComputerExamples/Paper3/Paper3/Manuscript_final'

In [11]:
df_final = pd.read_csv(parent_dir+'/CMIP6-Future_models_all_(with Sr-eq)_South America.csv')

In [12]:
df_final

,institution_id,source_id,table_id,grid_label,historical,ssp126,ssp245,ssp370,ssp585,Sr-100eq,Sr-400eq
0,CSIRO-ARCCSS,ACCESS-CM2,Amon,gn,1,1,1,1,1,116.575532,409.900705
1,CSIRO,ACCESS-ESM1-5,Amon,gn,1,1,0,0,1,95.728133,314.000824
2,AWI,AWI-CM-1-1-MR,Amon,gn,1,1,1,1,1,127.053403,446.708181
3,BCC,BCC-CSM2-MR,Amon,gn,1,1,1,1,1,93.549910,348.750239
4,CAS,CAS-ESM2-0,Amon,gn,1,1,1,1,1,120.418121,336.189765
5,NCAR,CESM2-WACCM,Amon,gn,1,1,1,1,1,164.953056,445.445854
6,CMCC,CMCC-CM2-SR5,Amon,gn,1,1,1,1,1,155.168020,288.220768
7,CMCC,CMCC-ESM2,Amon,gn,1,1,1,1,1,147.600075,282.739693
8,CCCma,CanESM5,Amon,gn,1,1,1,1,1,194.054560,337.462342
9,E3SM-Project,E3SM-1-1,Amon,gr,1,0,0,0,1,165.402306,338.410103


In [29]:
df_final['institution_id'].unique()

array(['CSIRO-ARCCSS', 'CSIRO', 'AWI', 'BCC', 'CAS', 'NCAR', 'CMCC',
       'CCCma', 'E3SM-Project', 'EC-Earth-Consortium', 'FIO-QLNM',
       'NOAA-GFDL', 'CCCR-IITM', 'INM', 'IPSL', 'NIMS-KMA', 'MIROC',
       'MPI-M', 'MRI', 'NUIST', 'NCC', 'AS-RCEC'], dtype=object)

In [ ]:
Sr

In [5]:
citation = pd.read_csv(parent_dir+'/CMIP6 Data References.csv')
citation

,SOURCE_ID,EXPERIMENT_ID,Data Reference
0,TaiESM1,NaN,"Tsai, I-Chun; Lee, Wei-Liang; Hsu, Huang-Hsiun..."
1,TaiESM1,hist-piNTCF,"Tsai, I-Chun; Lee, Wei-Liang; Hsu, Huang-Hsiun..."
2,TaiESM1,histSST,"Tsai, I-Chun; Lee, Wei-Liang; Hsu, Huang-Hsiun..."
3,TaiESM1,histSST-piNTCF,"Tsai, I-Chun; Lee, Wei-Liang; Hsu, Huang-Hsiun..."
4,TaiESM1,piClim-NTCF,"Tsai, I-Chun; Lee, Wei-Liang; Hsu, Huang-Hsiun..."
...,...,...,...
2771,UofMD,UofMD-landState-MAGPIE-ssp534-2-1-e,"Hurtt, George; Chini, Louise; Sahajpal, Ritvik..."
2772,UofMD,UofMD-landState-MAGPIE-ssp534-2-1-f,"Hurtt, George; Chini, Louise; Sahajpal, Ritvik..."
2773,UofMD,UofMD-landState-MAGPIE-ssp585-2-1-e,"Hurtt, George; Chini, Louise; Sahajpal, Ritvik..."
2774,UofMD,UofMD-landState-MAGPIE-ssp585-2-1-f,"Hurtt, George; Chini, Louise; Sahajpal, Ritvik..."


In [20]:
citation_final = df_final[['institution_id', 'source_id','historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']]

for Exp_id in ['historical','ssp126','ssp245','ssp370','ssp585']:
    citation_filtered = citation.where(citation.EXPERIMENT_ID == Exp_id).dropna()
    for source_id in df_final.source_id.unique():
        citation_filtered_2 = citation_filtered.where(citation_filtered.SOURCE_ID == source_id).dropna()
        #print(source_id, Exp_id)
        if citation_filtered_2['Data Reference'].empty:
            continue
        if citation_final.at[citation_final['source_id'].tolist().index(source_id), Exp_id] == 0:
            continue
        citation_final.at[citation_final['source_id'].tolist().index(source_id), Exp_id] = citation_filtered_2['Data Reference'].values[0]
        citation_filtered_2 = 0

/tmp/ipykernel_603/2767328175.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  citation_final.at[citation_final['source_id'].tolist().index(source_id), Exp_id] = citation_filtered_2['Data Reference'].values[0]
/tmp/ipykernel_603/2767328175.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  citation_final.at[citation_final['source_id'].tolist().index(source_id), Exp_id] = citation_filtered_2['Data Reference'].values[0]
/tmp/ipykernel_603/2767328175.py:12: SettingWithCopyWarning: 
A value is trying to b

In [23]:
citation_final.to_csv(parent_dir+'/Citation_final.csv')

In [ ]:
unique